### Hive assignment. Task2

Compare most popular tags in year 2016 with tags in 2009. Tag popularity is the number of questions (post_type_id=1) with this tag. Output top 10 tags in format:

```tag <tab> rank in 2016 <tab> rank in 2009 <tag> tag popularity in 2016 <tag> tag popularity in 2009```

Example:

```php 5 3 1234 5678```

For 'rank' in each year use rank() function. Order by 'rank in 2016'.

In [1]:
%%writefile query.hql

USE stackoverflow_;

SELECT
    SubQ16.tag, rnk16, rnk09, pop16, pop09
FROM (
    SELECT
        tag, popularity AS pop16, rank() OVER w AS rnk16
    FROM (
        SELECT
            year, tag, count(1) AS popularity
        FROM
            posts
        LATERAL VIEW
            explode(tags) posts AS tag
        WHERE
            year = 2016
            AND post_type_id=1
        GROUP BY
            year, tag
        ORDER BY
            popularity DESC
    ) AS SubQ
    WINDOW w AS (PARTITION BY year ORDER BY popularity DESC)
) AS SubQ16
LEFT JOIN (
    SELECT
        tag, popularity AS pop09, rank() OVER w AS rnk09
    FROM (
        SELECT
            year, tag, count(1) AS popularity
        FROM
            posts
        LATERAL VIEW
            explode(tags) posts AS tag
        WHERE
            year = 2009
            AND post_type_id=1
        GROUP BY
            year, tag
        ORDER BY
            popularity DESC
    ) AS SubQ
    WINDOW w AS (PARTITION BY year ORDER BY popularity DESC)
) AS SubQ09
ON
    SubQ16.tag = SubQ09.tag
ORDER BY
    rnk16
LIMIT 10;


Overwriting query.hql


In [2]:
! hive -f query.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
posts_sample_external
Time taken: 1.164 seconds, Fetched: 1 row(s)
OK
default
mskorokhod
Time taken: 0.027 seconds, Fetched: 2 row(s)
FAILED: SemanticException [Error 10072]: Database does not exist: stackoverflow_
